In [5]:
import time
import toml
import numpy as np
import matplotlib.pyplot as plt

from typing import Dict, Optional
from binance.client import Client
from datetime import datetime, timezone

from filter_conform import get_exchange_info_for_symbol

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 18})

In [6]:
cfg = toml.load("configuration.toml")
pkey = cfg["auth"]["pkey"]
skey = cfg["auth"]["skey"]

client = Client(pkey, skey)

## How to buy/sell a coin

Before anything else, the order has to conform to order filters, which can be retrieved 

In [7]:
from binance.client import Client

get_exchange_info_for_symbol(client, "DOGEUSDT")

{'symbol': 'DOGEUSDT',
 'status': 'TRADING',
 'baseAsset': 'DOGE',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00001000',
   'maxPrice': '1000.00000000',
   'tickSize': '0.00001000'},
  {'filterType': 'PERCENT_PRICE',
   'multiplierUp': '5',
   'multiplierDown': '0.2',
   'avgPriceMins': 5},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.10000000',
   'maxQty': '9000000.00000000',
   'stepSize': '0.10000000'},
  {'filterType': 'MIN_NOTIONAL',
   'minNotional': '10.00000000',
   'applyToMarket': True,
   'avgPriceMins': 5},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},


Important things in the exchange info:
- baseAssetPrecision / quoteAssetPrecision - neet to format orders to that precision
- filters - must conform order to filters

For constructing orders: I assume we have the desired quantity of USD / crypto that we want to buy or sell. For simplicity, take this to be a limit order at first. We can check that the order would pass the filters before sending it to binance. This would allow us to, in the future, modify orders before sending them in.